In [1]:
from torchvision import models
import torch

import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import matplotlib.pyplot as plt

resnet18_pretrained = models.resnet18(pretrained=True)

print(resnet18_pretrained)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
num_classes= 18
num_ftrs = resnet18_pretrained.fc.in_features
resnet18_pretrained.fc = nn.Linear(num_ftrs, num_classes)

In [3]:
class TrainDataset(Dataset):
    def __init__(self, img_paths, transform, json_data):
        self.img_paths = img_paths
        self.transform = transform
        self.json_data = json_data
        
    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        label = json_data[index//7]['img'][index%7]['class']
        
        return image, label

    def __len__(self):
        return len(self.img_paths)

In [4]:
import json
with open("train.json", "r") as f:
    json_data = json.load(f)
json_data = json_data['train']

In [5]:
# image_paths = []
# for image in json_data:
#     for img in image['img']:
#         print(img['path'])
#         image_paths.append(img['path'])

In [6]:
# image_paths[:10]

In [7]:
# meta 데이터와 이미지 경로를 불러옵니다.
path_dir = 'input/data/train/images'

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = []
for image in json_data:
    for img in image['img']:
        image_paths.append(img['path'])
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    transforms.RandomHorizontalFlip(0.5),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TrainDataset(image_paths, transform, json_data)

loader = DataLoader(
    dataset,
    batch_size=200, # 
    shuffle=True # shuffle 추가
)
from sklearn.metrics import f1_score
device = torch.device('cuda')
model = resnet18_pretrained.to(device)
model.train()

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

for epoch in range(10):
    running_loss = 0.0
    epoch_f1 = 0
    n_iter = 0
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # t_ = torch.zeros((100,18))
        # t_[range(100),labels]=1
        # print(t_)
        # t_ = t_.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:
            print('{},{:.5f} loss: {:.3f}'.format(epoch+1, i+1, running_loss/10))
            running_loss=0.0
        epoch_f1 += f1_score(preds.cpu().numpy(), labels.cpu().numpy(), average='macro')
        n_iter += 1
    print('Finished')
    print('{:.4f}'.format(epoch_f1))


1,10.00000 loss: 1.390
1,20.00000 loss: 0.654
1,30.00000 loss: 0.491
1,40.00000 loss: 0.444
1,50.00000 loss: 0.349
1,60.00000 loss: 0.323
1,70.00000 loss: 0.350
1,80.00000 loss: 0.317
1,90.00000 loss: 0.323
Finished
62.5199
2,10.00000 loss: 0.322
2,20.00000 loss: 0.267
2,30.00000 loss: 0.234
2,40.00000 loss: 0.227
2,50.00000 loss: 0.240
2,60.00000 loss: 0.239
2,70.00000 loss: 0.228
2,80.00000 loss: 0.221
2,90.00000 loss: 0.242
Finished
76.5163
3,10.00000 loss: 0.202
3,20.00000 loss: 0.190
3,30.00000 loss: 0.145
3,40.00000 loss: 0.168
3,50.00000 loss: 0.168
3,60.00000 loss: 0.125
3,70.00000 loss: 0.169
3,80.00000 loss: 0.153
3,90.00000 loss: 0.162
Finished
81.4185
4,10.00000 loss: 0.120
4,20.00000 loss: 0.137
4,30.00000 loss: 0.132
4,40.00000 loss: 0.141
4,50.00000 loss: 0.117
4,60.00000 loss: 0.108
4,70.00000 loss: 0.112
4,80.00000 loss: 0.125
4,90.00000 loss: 0.112
Finished
85.2702
5,10.00000 loss: 0.122
5,20.00000 loss: 0.108
5,30.00000 loss: 0.107
5,40.00000 loss: 0.079
5,50.00000 l

In [9]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [10]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir = '/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'batch_200_RandHorFlip_epoch_10.csv'), index=False)
print('test inference is done!')

test inference is done!


In [13]:
path_dir = 'input/data/train/images'

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = []
for image in json_data:
    for img in image['img']:
        image_paths.append(img['path'])
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TrainDataset(image_paths, transform, json_data)

loader = DataLoader(
    dataset,
    shuffle=True # shuffle 추가
)

device = torch.device('cuda')

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
model.eval()

all_predictions = []
all_labels = []
for images, labels in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

submission['ans'] = all_predictions
submission['label'] = all_labels

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'test.csv'), index=False)
print('test inference is done!')

ValueError: Length of values (18900) does not match length of index (12600)

In [11]:
import cv2

In [17]:
len(all_predictions)

18900

In [18]:
len(all_labels)

18900

In [31]:
pd.DataFrame(range(len(all_labels)),[all_predictions,all_labels])

0
0  0       0
1  1       1
15 15      2
4  4       3
   4       4
...      ...
0  0   18895
4  4   18896
3  3   18897
15 15  18898
10 10  18899

[18900 rows x 1 columns]

In [35]:
predab = pd.DataFrame([all_labels,all_predictions])

In [42]:
predab = predab.transpose()

In [49]:

for i in range(18900):
    if predab.iloc[i][0] != predab.iloc[i][1]:
        print(predab.iloc[i])

0    4
1    1
Name: 21, dtype: int64
0    16
1    13
Name: 29, dtype: int64
0    4
1    1
Name: 71, dtype: int64
0    14
1    13
Name: 80, dtype: int64
0    3
1    4
Name: 175, dtype: int64
0    8
1    7
Name: 216, dtype: int64
0    0
1    1
Name: 250, dtype: int64
0    3
1    0
Name: 341, dtype: int64
0    8
1    7
Name: 345, dtype: int64
0    10
1     7
Name: 406, dtype: int64
0    16
1    13
Name: 445, dtype: int64
0    14
1    13
Name: 463, dtype: int64
0    14
1    13
Name: 487, dtype: int64
0    0
1    1
Name: 511, dtype: int64
0    4
1    1
Name: 563, dtype: int64
0    16
1    13
Name: 594, dtype: int64
0    3
1    0
Name: 611, dtype: int64
0    4
1    1
Name: 641, dtype: int64
0    10
1     7
Name: 705, dtype: int64
0    4
1    1
Name: 750, dtype: int64
0    5
1    2
Name: 758, dtype: int64
0    4
1    1
Name: 869, dtype: int64
0    12
1    13
Name: 876, dtype: int64
0    12
1    13
Name: 892, dtype: int64
0    4
1    1
Name: 895, dtype: int64
0    16
1    13
Name: 916, dtype: 